In [29]:
import numpy as np
from numpy import linalg as LA

M = []
M.append(np.array(np.mat('1 1; 0 1'))) # x
M.append(np.array(np.mat('1 0; 1 1'))) # y
M.append(np.array(np.mat('1 -1; 0 1'))) # x-1
M.append(np.array(np.mat('1 0; -1 1'))) # y-1
F = []
F.append(np.array(np.mat('1 2; 0 1'))) # x
F.append(np.array(np.mat('1 0; 2 1'))) # y
F.append(np.array(np.mat('1 -2; 0 1'))) # x-1
F.append(np.array(np.mat('1 0; -2 1'))) # y-1

# Devuelve la lista de digitos de num en base,
# y la completa con ceros a la dcha hasta que
# tenga longitud length
# A la dcha xq digits te devuelve la lista al reves
def extended_digits(num, base, length):
    dig = num.digits(base = 4)
    dig_len = len(dig)
    if dig_len != length:
        # Completamos con ceros a la dcha
        padding = [0]*(length - dig_len)
        dig = dig + padding
    return dig

# Manera menos elegante del mundo de hacer remove
# de x*x^-1 y de y*y^-1
def remove_sub2(word, sublist):
    for i in range(len(word)-1):
        if word[i:i+2] == sublist:
            word = word[:i] + word[i+2:]
    return word      

# Quita los x*x^-1, y*y^-1, x^-1*x, y^-1*y de la palabra 
def reduce_w(word):
    word = remove_sub2(word, [0,2])
    word = remove_sub2(word, [2,0])
    word = remove_sub2(word, [1,3])
    word = remove_sub2(word, [3,1])
    return word


#Encuentra la matriz -1 en palabras de longitud length
def finder(length, goal = np.array(np.mat('-1 0; 0 -1'))):
    results = []
    for i in srange(4^length):
        mult = np.array(np.mat('1 0; 0 1'))
        b4_digits = extended_digits(i, 4, length)
        for digit in b4_digits:
            mult = np.matmul(mult, M[digit])
        if np.array_equal(mult, goal):
            results.append(b4_digits)
    return results

# Esta cosa de aqui es útil porque, dada una ristra de palabras,
# Te devuelve las palabras que, al quitarle los x*x-1, no han quedado
# con la longitud size-antiguas (o sea, que no son repes)
def nuevas_de_verdad(words, size_antiguas):
    reduced = [reduce_w(word) for word in words]
    reduced_n = [] # Metemos aqui los -1 que no son de longitud 6
    for word in reduced:
        if len(word) != size_antiguas:
            reduced_n.append(word)
    return reduced_n

def produce(length, gens = F, num_gens = 4):
    results = []
    for i in srange(num_gens^length):
        mult = np.array(np.mat('1 0; 0 1'))
        b4_digits = extended_digits(i, num_gens, length)
        for digit in b4_digits:
            mult = np.matmul(mult, gens[digit])
        results.append(mult)
    return results



def not_in(elem, lista):
    for elem2 in lista:
        if elem[0][0] == elem2[0][0] and elem[0][1] == elem2[0][1] and elem[1][0] == elem2[1][0] and elem[1][1] == elem2[1][1]:
            return False
    return True

def elim_dups(lista):
    final = []
    for elem in lista:
        if not_in(elem, final):
            final.append(elem)
    return final

def mod(lst, modulo=4):
    return [elem%modulo for elem in lst]

def mi_forma_bonita(mat):
    boolean = (mat[0][0]%4 == 1) and (mat[1][1]%4 == 1)
    boolean = boolean and (mat[0][1]%2 == 0) and (mat[1][0]%2 == 0)
    return boolean

def power(listadelistas, potencia):
    ret = []
    for clase in listadelistas:
        ret.append([matriz^potencia for matriz in clase])
    return ret

In [2]:
l = finder(6) #6 es la length minima donde aparece el -1


In [4]:
a8 = finder(8)   
a_reduced_8 = nuevas_de_verdad(a8, 6)

In [5]:
# Investigando saca de a_reduced_8 las palabras que son conjugadas de un generador o de su inverso:
# gen*algo*gen^-1
# Resulta que 'algo' es las cosas bonitas de tamanio 6 que son -1 (los elementos de l)
# Y que no es casualidad: gen*bonito*gen^-1 = gen*-1*gen^-1 = -1 xq -1 esta en el centro!

investigando = []
for word in a_reduced_8:
    if (word[0] == 0 and word[-1] == 2) or (word[0] == 2 and word[-1] == 0):
        investigando.append(word)
    if (word[0] == 1 and word[-1] == 3) or (word[0] == 3 and word[-1] == 1):
        investigando.append(word)
#len(investigando), investigando  

In [46]:
# Aqui meto todas las demas (las que no son conjugadas)
lasquequedan = [word for word in a_reduced_8 if word not in investigando]
len(lasquequedan)

32

In [47]:
# De las que quedan, meto aqui las que empiezan y acaban por el mismo generador 
# (Son una cosa, y la misma cosa dada la vuelta!!)
[word for word in lasquequedan if word[0] == word[-1]]

[[0, 3, 0, 1, 0, 3, 0, 0],
 [0, 0, 3, 0, 1, 0, 3, 0],
 [1, 2, 1, 0, 1, 2, 1, 1],
 [1, 1, 2, 1, 0, 1, 2, 1],
 [2, 2, 1, 2, 3, 2, 1, 2],
 [2, 1, 2, 3, 2, 1, 2, 2],
 [3, 3, 0, 3, 2, 3, 0, 3],
 [3, 0, 3, 2, 3, 0, 3, 3]]

In [44]:
# Y aqui todo lo demas
# Hay algunas que estan dadas la vuelta, y otras que mantienen los endpoints e invierten el orden de los 3 pares internos
# Veanse los 2 primeros elementos
#      0  00 30 10  3
#      0  10 30 00  3
sorted([word1 for word1 in lasquequedan if word1 not in [word for word in lasquequedan if word[0] == word[-1]]])        

[[0, 0, 0, 3, 0, 1, 0, 3],
 [0, 1, 0, 3, 0, 0, 0, 3],
 [0, 1, 2, 1, 1, 1, 2, 1],
 [0, 3, 0, 0, 0, 3, 0, 1],
 [0, 3, 2, 3, 0, 3, 3, 3],
 [0, 3, 3, 3, 0, 3, 2, 3],
 [1, 0, 1, 2, 1, 1, 1, 2],
 [1, 0, 3, 0, 0, 0, 3, 0],
 [1, 1, 1, 2, 1, 0, 1, 2],
 [1, 2, 1, 1, 1, 2, 1, 0],
 [1, 2, 2, 2, 1, 2, 3, 2],
 [1, 2, 3, 2, 1, 2, 2, 2],
 [2, 1, 0, 1, 2, 1, 1, 1],
 [2, 1, 1, 1, 2, 1, 0, 1],
 [2, 1, 2, 2, 2, 1, 2, 3],
 [2, 2, 2, 1, 2, 3, 2, 1],
 [2, 3, 0, 3, 3, 3, 0, 3],
 [2, 3, 2, 1, 2, 2, 2, 1],
 [3, 0, 0, 0, 3, 0, 1, 0],
 [3, 0, 1, 0, 3, 0, 0, 0],
 [3, 0, 3, 3, 3, 0, 3, 2],
 [3, 2, 1, 2, 2, 2, 1, 2],
 [3, 2, 3, 0, 3, 3, 3, 0],
 [3, 3, 3, 0, 3, 2, 3, 0]]

In [2]:
# res1 tiene las libres
res1 = []
for i in range(8):
    aux = produce(i)
    for mat in aux:
        res1.append(mat)
res1 = elim_dups(res1)

# res2 tiene todas
res2 = []
for i in range(10):
    aux = produce(i, gens = M)
    for mat in aux:
        res2.append(mat)
res2 = elim_dups(res2)
print 'done3'

done3


In [3]:
not_free = []
for mat in res2:
    if not_in(mat, res1):
        not_free.append(mat)
print 'done4' # not_free tiene las no libres

not_free = elim_dups(not_free)


done4


In [16]:
f = open('libres_mod_4', 'w')
string = ''
for mat in res1:
    if mi_forma_bonita(mat) == False:
        print 'Eres idiota2'
    string += str(mod(mat[0], 4)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 4)) + '\t\t\t' + str(mat[1]) + '\n\n'

f.write(string)
f.close()

# Parece q si eres libre, no puedes ser 3 modulo 4!

f = open('no_libres_mod_4', 'w')
string = ''
for mat in not_free:
    if mi_forma_bonita(mat) == True:
        print 'Eres idiota'
    string += str(mod(mat[0], 4)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 4)) + '\t\t\t' + str(mat[1]) + '\n\n'

f.write(string)
f.close()

# No es suficiente.
# Lloro un pelin na mas
# A lo mejor si q es suficiente! 
# No entiendo muy bien xq, pero mirando libres de longitud 6, pa mi q estaba
# mirando matrices de longitud 12 y por tanto ya habia cubierto las de 
# longitud 10, pero parece q no.
# Las normales de longitud 10 que no tenian ningun 3 en su modulo y no estaban
# dentro de las libres de longitud 6, si estan dentro de las libres
# de longitud 7. wtf

In [58]:
# Parese qpa esto vamos a pasar de numpy :/
Z4 = Integers(4)
M4 = MatrixSpace(Z4,2,2)
SLZ4 = []
for matrix in M4:
    if det(matrix) == 1:
        SLZ4.append(matrix)

# image of the free group under the hom: SLZ -> SLZ4        
C = []
C.append(M4([1,0,0,1]))
C.append(M4([1,0,2,1]))
C.append(M4([1,2,0,1]))
C.append(M4([1,2,2,1]))

In [59]:
def not_in_2(clase, clases):
    elem = clase[0]
    for elems in clases:
        if elem in elems:
            return False
    return True

clases = []
for matrix in SLZ4:
    clase = [(matrix * f)for f in C]
    if not_in_2(clase, clases):
        clases.append(clase)

ordenes = []
for clase in clases:
    representante = clase[0]
    aux = representante
    orden = 0
    while 1:
        orden += 1
        if aux in clases[0]: #si representante dentro de la clase identidad
            ordenes.append(orden)
            break
        else:
            aux *= representante
ordenes

[1, 2, 2, 4, 4, 3, 6, 6, 6, 2, 4, 2]

In [80]:
len(SLZ4)

48

Ya tengo la prueba de que $\overline{\mathcal{M}} = \varphi^{-1}(\varphi(\mathcal{M})*\varphi(\mathbb{S}))$!!

Posibilidades: 
* $Dic_3$ (ayer parecia que no): $a^6=1, b^2=a^3, aba=b$
* $Dih_6$ (creo que este es el diedrico): $a^6=b^2=1, aba=b$
* $A_4$ : no, porque hay subgrupos de orden 6 :D

In [90]:
o6 = [clases[6], clases[7], clases[8]]
o4 = [clases[3], clases[4], clases[10]]
o2 = [clases[1], clases[2], clases[11]]

def mul(la,lb):
    ret = []
    for a in la:
        for b in lb:
            c = a*b
            c.set_immutable()
            ret.append(c)
    return Set(ret)
def inv(la):
    return [a^-1 for a in la]
i=0
for clase1 in clases:
    for clase2 in clases:
        if mul(clase1, clase2) != mul(clase2, clase1):
            i+=1
            #print len(mul(clase1, clase2)), len(mul(clase1, clase2))
            #print '1*2'
            #for elem in mul(clase1, clase2):
            #    print elem, '\n'
            #print '2*1'
            #for elem in mul(clase2, clase1):
            #    print elem, '\n'
for n in C:
    for m in SLZ4:
        if m*n*(m^(-1)) not in C:
            print m, '\n', m^(-1),'\n\n', n, '\n\n', m*n*m^(-1) 
            print 'not normal'
            

104
[1 1]
[0 1] 
[1 3]
[0 1] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[2 1]
[1 1] 
[1 3]
[3 2] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[1 3]
[0 1] 
[1 1]
[0 1] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[0 3]
[1 1] 
[1 1]
[3 0] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[0 1]
[3 1] 
[1 3]
[1 0] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[3 1]
[2 1] 
[1 3]
[2 3] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[3 1]
[0 3] 
[3 3]
[0 3] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[2 1]
[1 3] 
[3 3]
[3 2] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[1 1]
[2 3] 
[3 3]
[2 1] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[0 3]
[1 3] 
[3 1]
[3 0] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[0 1]
[3 3] 
[3 3]
[1 0] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[3 3]
[2 1] 
[1 1]
[2 3] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[3 3]
[0 3] 
[3 1]
[0 3] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[2 3]
[3 1] 
[1 1]
[1 2] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[1 3]
[2 3] 
[3 1]
[2 1] 

[1 0]
[2 1] 

[3 2]
[2 3]
not normal
[2 3]
[3 3] 
[3 1]
[1 2] 

[1 0]
[2 

In [79]:
a = clases[0]
b = clases[1]
todos = []
for m1 in a:
    for m2 in b:
        c = m1*m2
        c.set_immutable()
        todos.append(c)
set(todos)

{[1 1]
 [0 1], [1 1]
 [2 3], [1 3]
 [0 1], [1 3]
 [2 3], [3 1]
 [0 3], [3 1]
 [2 1], [3 3]
 [0 3], [3 3]
 [2 1]}

In [88]:
print C[1]*C[2], '\n', C[3]

[1 2]
[2 1] 
[1 2]
[2 1]
